In [134]:
# Jupyter NoteBook pour la question 2 du devoir 2 IFT3295
# Fait par Maggie Robert & Louis-André Brassard
# Faire bien attention de rouler le notebook d'un coup dans l'ordre pour éviter
# que certaines listes ne soit modifier plus qu'elles le doivent

In [135]:
import numpy as np
import blosum as bl
from itertools import combinations

In [136]:
def read_fasta(fp):
    name, seq = None, []
    for line in fp:
        line = line.rstrip()
        if line.startswith(">"):
            if name: yield (name, ''.join(seq))
            name, seq = line, []
        else:
            seq.append(line)
    if name: yield (name, ''.join(seq))

In [137]:
# read fasta sequences
reads = []
names = []
with open('sequences.fasta') as fp:
    for (name, seq) in read_fasta(fp):
        names.append(name)
        reads.append(seq)



In [138]:
# create dictionary from Blosum62 matrix from https://www.biostars.org/p/405990/
# bd.get('AA') to get the score wanted
mat = bl.BLOSUM(62)
bd = dict(mat)

In [139]:
def score(c1, c2):
    return int(bd.get(c1 + c2))

In [140]:
def alignement_global(s1, s2):
    a = -10
    b = -1

    M = np.ones(shape=(len(s2) + 1, len(s1) + 1)) * np.NINF
    X = np.ones(shape=(len(s2) + 1, len(s1) + 1)) * np.NINF
    Y = np.ones(shape=(len(s2) + 1, len(s1) + 1)) * np.NINF

    # Initial conditions
    M[0][0] = 0

    for i in range(len(s2) + 1):
        X[i][0] = a + b * i

    for j in range(len(s1) + 1):
        Y[0][j] = a + b * j

    for row in range(1, len(s2) + 1):
        for col in range(1, len(s1) + 1):
            character1 = s1[col - 1]
            character2 = s2[row - 1]
            sim_score = score(character1, character2)

            # M[row][col] = max(M[row - 1][col - 1] + sim_score, X[row - 1][col - 1] + sim_score,
            #                   Y[row - 1][col - 1] + sim_score)
            # X[row][col] = max(X[row - 1][col] + b, M[row - 1][col] + a + b)
            # Y[row][col] = max(Y[row][col - 1] + b, M[row][col - 1] + a + b)

            X[row][col] = max((a + b + M[row][col - 1]), (b + X[row][col - 1]), (a + b + Y[row][col - 1]))
            Y[row][col] = max((a + b + M[row - 1][col]), (a + b + X[row - 1][col]), (b + Y[row - 1][col]))
            M[row][col] = max((sim_score + M[row - 1][col - 1]), (X[row][col]), (Y[row][col]))

    return [M, X, Y]

In [141]:
def sim_matrix():
    m = np.zeros(shape=(len(reads), len(reads)))
    for i in range(len(reads)):
        for j in range(len(reads)):
            v = alignement_global(reads[i], reads[j])
            v1 = v[0][len(reads[j])][len(reads[i])]
            v2 = v[1][len(reads[j])][len(reads[i])]
            v3 = v[2][len(reads[j])][len(reads[i])]
            m[i][j] = max(v1, v2, v3)

    print(m)
    return m


In [142]:
def seq_centrale(m):
    total = []
    for row in m:
        sum = 0
        for s in row:
            sum += s

        total.append(sum)

    index = 0
    min_value = 99999999999
    for i in range(len(total)):
        if total[i] < min_value:
            min_value = total[i]
            index = i

    print('La séquence centrale est ' + 'S' + str(index + 1))
    print(min_value)
    print(total)

In [143]:
def backtrace(s1,s2, M,X,Y):
    seq1 = ''
    seq2 = ''

    i = len(s2)
    j = len(s1)

    insertion =[]

    while i > 0 or j > 0:
        if i > 0 and j > 0 and M[i][j] == M[i-1][j-1] + score(s2[i - 1],s1[j - 1]) :
            seq1 += s1[j-1]
            seq2 += s2[i-1]
            i -= 1
            j -= 1
        elif i > 0 and M[i][j] == Y[i][j]:
            seq1 += '*'
            seq2 += s2[i-1]
            i -= 1
            insertion.insert(0,i)
        elif j > 0 and M[i][j] == X[i][j]:
            seq1 += s1[j-1]
            seq2 += '*'
            j -= 1

    seq1r = ''.join([seq1[j] for j in range(-1, -(len(seq1) + 1), -1)])
    seq2r = ''.join([seq2[j] for j in range(-1, -(len(seq2) + 1), -1)])

    return [seq1r, seq2r,insertion]

In [144]:
# Find similarity matrix
sim_mat = sim_matrix()

# Find central sequence
seq_centrale(sim_mat)

[[808.  54.  20.  34.  80.]
 [ 54. 983.  94. 143.  77.]
 [ 20.  94. 737.  86.  16.]
 [ 34. 143.  86. 809.  20.]
 [ 80.  77.  16.  20. 772.]]
La séquence centrale est S3
953.0
[996.0, 1351.0, 953.0, 1092.0, 965.0]


In [145]:
def add_indels(word_list, ll):
    results = []
    for s in word_list:
        inv = len(s)
        for i in ll:
            if i > inv:
                s = s + '*'
            else:
                s = s[:int(i)] + '*' + s[int(i):]
        results.append(s)
    return results

In [146]:
# s3 - s1 alignement
star_alignment = []
central = reads[2]
[M, X, Y] = alignement_global(central, reads[0])
c, s, insertions = backtrace(central, reads[0], M, X, Y)
central = c
star_alignment.append(s)
print(*star_alignment, sep="\n")

MGEIGFTEKQEALVKESWEILKQDIPKYSLHFFSQILEIAPAAKGLFSFLRDSDEVPHNNPKLKAHAVKVFKMTCETAIQLREEGKVVVADTTLQYLGSIHLKSGVIDP*HFEVVKEALLRTLKEGLGEKYNEEVEGAWSQAYDHLALAIKTEMKQEES


In [147]:
# s3 -s2 alignment
[M, X, Y] = alignement_global(central, reads[1])
c, s , insertions = backtrace(central, reads[1], M, X, Y)
central = c
# add new indels to s1
star_alignment = add_indels(star_alignment, insertions)
star_alignment.append(s)
print(*star_alignment, sep="\n")
print(central)

M***************GEIGFTEKQEALVKESWEILKQDIPKYSLHFFSQILEIAPAAKGLFSFLRDS*DEV*PHNNPKLKAHAVKVFKMTCETAIQLREEGKVVVADTTLQYLGSIHLKSGVIDP*HFEVVKEALLRTLKEGLGEKYNEEVEGAWSQAYDHLALAIKTEMKQEES***********************
MEKVPGEMEIERRERSEELSEAERKA**VQATWARLYANCEDVGVAILVRFFVNFPSAKQYFSQFKHMEEPLEMERSPQLRKHACRVMGALNTVVENL**HDPEKVSS*VLSLVGKAHALKHKVEPVYFKILSGVILEVIAEEFANDFPPETQ****RAWAKLRGLIYSHVTAAYKEVGWVQQVPNATTPPATLPSSGP
M****************VLSAADKNN**VKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHF****D*LSH*GSAQIKGHGKKVVAALIEAANHI**DD***IAG*TLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVH***A*SLDKFLCAVGTVLTAKYR***********************


In [148]:
# s3 -s4 alignment
[M, X, Y] = alignement_global(central, reads[3])
c, s, insertions = backtrace(central, reads[3], M, X, Y)
central = c
# add indels to s1 and s2
star_alignment = add_indels(star_alignment,insertions)
star_alignment.append(s)
print(*star_alignment, sep="\n")
print(central)

M***************GEIGFTEKQEALVKESWEILKQDIPKYSLHFFSQILEIAPAAKGLFSFLRDS*DEV****PHNNPKLKAHAVKVFKMTCETAIQLREEGKVVVADTTLQYLGSIHLKSGVIDP*HFEVVKEALLRTLKEGLGEKYNE*EVEGAWSQAYDHLALAIKTEMKQEES***********************
MEKVPGEMEIERRERSEELSEAERKA**VQATWARLYANCEDVGVAILVRFFVNFPSAKQYFSQFKHMEEPLE***MERSPQLRKHACRVMGALNTVVENL**HDPEKVSS*VLSLVGKAHALKHKVEPVYFKILSGVILEVIAEEFANDFPP*ETQ****RAWAKLRGLIYSHVTAAYKEVGWVQQVPNATTPPATLPSSGP
M******************GLSDGEWQLVLNVWGKVEADIPGHGQEVLIRLFKGHPETLEKFDKFKHLKSEDEMKASEDLKKHGATVLTALGGILKKKGH*********HEAEIKPLAQSHATKHKIPVKYLEFISECIIQVLQSKHPGDFGADAQ****GAMNKALELFRKDMASNYK******************ELGFQG
M****************VLSAADKNN**VKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHF****D*LSH*GSAQIKGHGKKVVAAL***IEAANHI**DD***IAG*TLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVH***A*SLDKFLCAVGTVLTAKYR************************


In [149]:
# s3 -s5 alignment
[M, X, Y] = alignement_global(central, reads[4])
c, s, insertions = backtrace(central, reads[4], M, X, Y)
central = c
# add new indels to s1 s2 s4
star_alignment = add_indels(star_alignment,insertions)
star_alignment.append(s)
print(*star_alignment, sep="\n")

M***************GEIGFTEKQEALVKESWEIL***KQDIPKYS*LHFFSQILEIAPAAKGLFSFLRDS*DEV****PHNNPKL****KAH*AV*KVFKMTCETAIQLREEGKVVVADTTLQYLGSIHLKSGVIDP*HFEVVKEALLRTLKEGLGEKYNE*EVEGAWSQAYDHLALAIKTEMKQEES***********************
MEKVPGEMEIERRERSEELSEAERKA**VQATWARL***YANCEDVG*VAILVRFFVNFPSAKQYFSQFKHMEEPLE***MERSPQL****RKH*AC*RVMGALNTVVENL**HDPEKVSS*VLSLVGKAHALKHKVEPVYFKILSGVILEVIAEEFANDFPP*ETQ****RAWAKLRGLIYSHVTAAYKEVGWVQQVPNATTPPATLPSSGP
M******************GLSDGEWQLVLNVWGKV***EADIPGHG*QEVLIRLFKGHPETLEKFDKFKHLKSEDEMKASEDLKKH****GAT*VL*TALGGILKKKGH*********HEAEIKPLAQSHATKHKIPVKYLEFISECIIQVLQSKHPGDFGADAQ****GAMNKALELFRKDMASNYK******************ELGFQG
M********************ERLESELIRQSWRAVSRSPLEHGTVLFSRLFALEPSLLPLFQYNGRQFSSPEDCLS***SPEFLDHIRKVM**LV**IDAA*****VTN**VED**LSSLEEYLATLGRKHRAVGVRLSSFSTVGES*LLYM*L**EKCLGPDFTP**ATR*TAWSQLYGAVVQAMSR*******************GWDGE


In [150]:
star_alignment.insert(2,central)
for i in star_alignment:
    print(i.replace('*','-'))

M---------------GEIGFTEKQEALVKESWEIL---KQDIPKYS-LHFFSQILEIAPAAKGLFSFLRDS-DEV----PHNNPKL----KAH-AV-KVFKMTCETAIQLREEGKVVVADTTLQYLGSIHLKSGVIDP-HFEVVKEALLRTLKEGLGEKYNE-EVEGAWSQAYDHLALAIKTEMKQEES-----------------------
MEKVPGEMEIERRERSEELSEAERKA--VQATWARL---YANCEDVG-VAILVRFFVNFPSAKQYFSQFKHMEEPLE---MERSPQL----RKH-AC-RVMGALNTVVENL--HDPEKVSS-VLSLVGKAHALKHKVEPVYFKILSGVILEVIAEEFANDFPP-ETQ----RAWAKLRGLIYSHVTAAYKEVGWVQQVPNATTPPATLPSSGP
M----------------VLSAADKNN--VKGIFTKIAGHAEEYGAETLERMFTTYP---PTKTYFPH-F----D-LSH-GSAQIKGHGKKVVAAL---IEAANHI--DD---IAG-TLSKLSD----LHA-HK-LRVDPVNFKLLGQCFLVVVAIHHPAALTPEVH---A-SLDKFLCAVGTVLTAKYR------------------------
M------------------GLSDGEWQLVLNVWGKV---EADIPGHG-QEVLIRLFKGHPETLEKFDKFKHLKSEDEMKASEDLKKH----GAT-VL-TALGGILKKKGH---------HEAEIKPLAQSHATKHKIPVKYLEFISECIIQVLQSKHPGDFGADAQ----GAMNKALELFRKDMASNYK------------------ELGFQG
M--------------------ERLESELIRQSWRAVSRSPLEHGTVLFSRLFALEPSLLPLFQYNGRQFSSPEDCLS---SPEFLDHIRKVM--LV--IDAA-----VTN--VED--LSSLEEYLATLGRKHRAVGVRLSSFST

In [151]:
# Give the SP score based on similarity
def scoreSP(central_align):
    comb = combinations(central_align, 2)
    total = 0
    for c in comb:
        for i in range(len(c[0])):
            x = c[0][i]
            y = c[1][i]
            if c[0][0] == '*' or c[1][0] == '*' and c[0][0] != c[1][0]:
                total += -11

            elif c[0][i] == '*' and c[1][i] == '*':
                total += 0

            # Ouverture gap en s1
            elif c[0][i] == '*' and c[0][i-1] != '*' :
                total += -11

            # ouverture gap en s2
            elif c[1][i] == '*' and c[1][i-1] != '*' :
                total += -11

            #extension gap s1
            elif c[0][i] == '*' and c[0][i-1] == '*':
                total += -1

            # extension gap s2
            elif c[1][i] == '*' and c[1][i-1] == '*':
                total += -1
            else:
                total += score(c[0][i], c[1][i])
    return total
scoreSP(star_alignment)

-1818

In [152]:
# Give the concensus alignment Z

def concessus(ca):
    Z = ''
    for i in range(len(ca[0])):
        if ca[0][i] == ca[1][i] == ca[2][i] == ca[3][i] == ca[4][i]:
            Z += ca[0][i]
        else:
            Z += '-'
    return Z
concessus(star_alignment)

'M----------------------------------------------------------P-------------------------------------*-------------------------------------------------------------------------------------------------------------------'